In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Cấu hình hiển thị
pd.set_option('display.max_columns', None)
print("import thành công")

import thành công


# 2. TẢI DỮ LIỆU

In [4]:
file_path = '../data/raw/superstore.csv' 
try:
    # Nếu chạy lỗi file path
    df = pd.read_csv(file_path)
    print(f"Đã tải dữ liệu thành công. Kích thước: {df.shape}")
except FileNotFoundError:
    print("Không tìm thấy file. Vui lòng kiểm tra lại đường dẫn.")
    # test nhanh
    df = pd.read_csv('superstore.csv')

Đã tải dữ liệu thành công. Kích thước: (51290, 27)


# 3. Tổng Quan

In [5]:
print("\n--- 3.1. THÔNG TIN CẤU TRÚC (INFO) ---")
print(df.info())

print("\n--- 3.2. MẪU DỮ LIỆU (HEAD) ---")
display(df.head(3))


--- 3.1. THÔNG TIN CẤU TRÚC (INFO) ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Category        51290 non-null  object 
 1   City            51290 non-null  object 
 2   Country         51290 non-null  object 
 3   Customer.ID     51290 non-null  object 
 4   Customer.Name   51290 non-null  object 
 5   Discount        51290 non-null  float64
 6   Market          51290 non-null  object 
 7   记录数             51290 non-null  int64  
 8   Order.Date      51290 non-null  object 
 9   Order.ID        51290 non-null  object 
 10  Order.Priority  51290 non-null  object 
 11  Product.ID      51290 non-null  object 
 12  Product.Name    51290 non-null  object 
 13  Profit          51290 non-null  float64
 14  Quantity        51290 non-null  int64  
 15  Region          51290 non-null  object 
 16  Row.ID          51290 non-null  int6

,Category,City,Country,Customer.ID,Customer.Name,Discount,Market,记录数,Order.Date,Order.ID,Order.Priority,Product.ID,Product.Name,Profit,Quantity,Region,Row.ID,Sales,Segment,Ship.Date,Ship.Mode,Shipping.Cost,State,Sub.Category,Year,Market2,weeknum
0,Office Supplies,Los Angeles,United States,LS-172304,Lycoris Saunders,0.0,US,1,2011-01-07 00:00:00.000,CA-2011-130813,High,OFF-PA-10002005,Xerox 225,9.3312,3,West,36624,19,Consumer,2011-01-09 00:00:00.000,Second Class,4.37,California,Paper,2011,North America,2
1,Office Supplies,Los Angeles,United States,MV-174854,Mark Van Huff,0.0,US,1,2011-01-21 00:00:00.000,CA-2011-148614,Medium,OFF-PA-10002893,"Wirebound Service Call Books, 5 1/2"" x 4""",9.2928,2,West,37033,19,Consumer,2011-01-26 00:00:00.000,Standard Class,0.94,California,Paper,2011,North America,4
2,Office Supplies,Los Angeles,United States,CS-121304,Chad Sievert,0.0,US,1,2011-08-05 00:00:00.000,CA-2011-118962,Medium,OFF-PA-10000659,"Adams Phone Message Book, Professional, 400 Me...",9.8418,3,West,31468,21,Consumer,2011-08-09 00:00:00.000,Standard Class,1.81,California,Paper,2011,North America,32


# 4. Kiểm tra chất lượng data 

In [6]:
print("\n--- 4.1. GIÁ TRỊ THIẾU (MISSING VALUES) ---")
missing = df.isnull().sum()
missing = missing[missing > 0]
if missing.empty:
    print("Không có giá trị thiếu (Null) nào.")
else:
    print("Các cột sau có dữ liệu thiếu:")
    print(missing)
    print("-> Cần xử lý trong bước Cleaning (Điền mặc định hoặc Xóa).")


--- 4.1. GIÁ TRỊ THIẾU (MISSING VALUES) ---
Không có giá trị thiếu (Null) nào.


In [7]:
# 4.2. Kiểm tra Trùng lặp (Duplicates)
print("\n--- 4.2. DỮ LIỆU TRÙNG LẶP (DUPLICATES) ---")
# Check 1: Trùng lặp hoàn toàn (Full Row Duplicates)
dup_rows = df.duplicated().sum()
print(f"- Số dòng trùng lặp hoàn toàn: {dup_rows}")


--- 4.2. DỮ LIỆU TRÙNG LẶP (DUPLICATES) ---
- Số dòng trùng lặp hoàn toàn: 0


In [8]:
# Check 2: Trùng lặp Business Key (Order ID + Product ID)
# Một Order không nên có 2 dòng cho cùng 1 Product ID (trừ khi tách shipping)
business_dups = df.duplicated(subset=['Order.ID', 'Product.ID']).sum()
print(f"- Số dòng trùng lặp Business Key (Order ID + Product ID): {business_dups}")

if business_dups > 0:
    print("-> Có khả năng lỗi nhập liệu. Cần gộp.")

- Số dòng trùng lặp Business Key (Order ID + Product ID): 38
-> Có khả năng lỗi nhập liệu. Cần gộp.


In [9]:
# 4.3. Kiểm tra Logic Thời gian (Time Consistency)
print("\n--- 4.3. LOGIC THỜI GIAN (TIME LOGIC) ---")
# Chuyển đổi tạm thời sang datetime để check
df['Order.Date.Temp'] = pd.to_datetime(df['Order.Date'])
df['Ship.Date.Temp'] = pd.to_datetime(df['Ship.Date'])

# Logic: Ngày giao hàng < Ngày đặt hàng?
invalid_dates = df[df['Ship.Date.Temp'] < df['Order.Date.Temp']]
num_invalid_dates = len(invalid_dates)
print(f"- Số đơn hàng có Ngày Ship < Ngày Đặt: {num_invalid_dates}")
if num_invalid_dates > 0:
    print("-> ⚠️ Dữ liệu thời gian bị sai logic. Cần xóa hoặc sửa.")

# Xóa cột tạm
df.drop(columns=['Order.Date.Temp', 'Ship.Date.Temp'], inplace=True)


--- 4.3. LOGIC THỜI GIAN (TIME LOGIC) ---
- Số đơn hàng có Ngày Ship < Ngày Đặt: 0


# 5. THỐNG KÊ MÔ TẢ & PHÁT HIỆN BẤT THƯỜNG 

In [10]:
print("\n--- 5. THỐNG KÊ SỐ   ---")
# Chỉ lấy các cột số quan trọng
numeric_cols = ['Sales', 'Quantity', 'Discount', 'Profit', 'Shipping.Cost']
print(df[numeric_cols].describe())


--- 5. THỐNG KÊ SỐ   ---
              Sales      Quantity      Discount        Profit  Shipping.Cost
count  51290.000000  51290.000000  51290.000000  51290.000000   51290.000000
mean     246.498440      3.476545      0.142908     28.610982      26.375818
std      487.567175      2.278766      0.212280    174.340972      57.296810
min        0.000000      1.000000      0.000000  -6599.978000       0.002000
25%       31.000000      2.000000      0.000000      0.000000       2.610000
50%       85.000000      3.000000      0.000000      9.240000       7.790000
75%      251.000000      5.000000      0.200000     36.810000      24.450000
max    22638.000000     14.000000      0.850000   8399.976000     933.570000


In [11]:
# Check nhanh các giá trị âm vô lý (Sales, Quantity không được âm)
neg_sales = (df['Sales'] < 0).sum()
neg_qty = (df['Quantity'] < 0).sum()
print(f"\n- Số dòng Sales âm: {neg_sales}")
print(f"- Số dòng Quantity âm: {neg_qty}")


- Số dòng Sales âm: 0
- Số dòng Quantity âm: 0


In [12]:
# Check lợi nhuận âm (Loss makers) - Đây không phải lỗi, mà là Insight
loss_makers = (df['Profit'] < 0).sum()
print(f"- Số dòng Lợi nhuận âm (Lỗ): {loss_makers} (Chiếm {loss_makers/len(df):.1%} dữ liệu)")

- Số dòng Lợi nhuận âm (Lỗ): 12544 (Chiếm 24.5% dữ liệu)
